In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame()

In [3]:
from logger import Logger

In [4]:
loggerDf = Logger('LOGS_finetuning.csv', mode='df')

In [5]:
import difflib
def show_diff(text, n_text):
    """
    http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    seqm = difflib.SequenceMatcher(None, text, n_text)
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<font color=red>^" + seqm.b[b0:b1] + "</font>")
        elif opcode == 'delete':
            output.append("<font color=blue>^" + seqm.a[a0:a1] + "</font>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            output.append("<font color=green>^" + seqm.b[b0:b1] + "</font>")
        else:
            raise RuntimeError("unexpected opcode")
    return ''.join(output)

In [17]:
print(loggerDf.logsFile.loc[33]['cfg'])

from model import MainModel
from torch import optim
import torch.nn as nn
import os
from logger import Logger
from libs.earlystopping import EarlyStopping

import torch, random
import numpy as np

def seed_torch(seed=13):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


class Config:
    def __init__(self, batch_size=32, lr=0.00015, p_horizontalflip=0.4, model_type='ResNet101', training_mode='only_new'):
        ## INFO ABOUT EXPERIMENT
        self.logsFileName = 'LOGS'
        self.logsFileName_finetuning = 'LOGS_finetuning'
        self.seed = 13

        seed_torch(self.seed)

        if os.path.exists('./Logs/' + self.logsFileName + '.csv'):
            if training_mode == 'only_new':
                self.df_logger = Logger

In [13]:
loggerDf.logsFile.tail(10).sort_values(by='best-valid-loss')

,date,data-type,net-architecture,loss-func,optim,scheduler,early-stopping-patience,parameters-amount,n-epochs,batch-size,best-train-loss,best-valid-loss,best-kappa,lb-kappa-score,cfg,dataset,trainloop,time_estimated
31,2019.08.31 06:30:07,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,28342833.0,20.0,16.0,0.415068,0.359604,0.7979,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,12374.71
32,2019.08.31 13:23:42,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,28342833.0,28.0,16.0,0.406893,0.379268,0.8079,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,17892.55
30,2019.08.30 23:05:50,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,28342833.0,18.0,16.0,0.499034,0.386216,0.8080,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,11172.06
33,2019.09.01 00:22:48,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,28342833.0,18.0,16.0,0.493762,0.406982,0.8013,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,11152.56
29,2019.08.30 18:52:03,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,28342833.0,20.0,16.0,0.430988,0.420640,0.7764,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,12441.74
34,2019.09.01 11:38:45,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,28342833.0,34.0,16.0,0.445161,0.429182,0.7895,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,20825.01
28,2019.08.30 16:16:40,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,6514465.0,11.0,16.0,0.955389,1.860145,0.0057,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,2826.17
36,2019.09.02 07:50:38,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,42502209.0,200.0,16.0,0.967711,1.931307,-0.0001,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,NaN
37,2019.09.02 07:57:36,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...,10.0,42502209.0,18.0,16.0,0.939368,1.935164,-0.0004,NaN,from model import MainModel\nfrom torch import...,# from retinal_process import *\n# from utils_...,\nimport numpy as np # linear algebra\nimport ...,6573.38
35,2019.09.02 07:26:01,new_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.ReduceL

In [19]:
loggerDf.logsFile.loc[3, 'data-type'] = 'finetuning_new'
loggerDf.logsFile.loc[3, 'n-epochs'] = 50
loggerDf.logsFile.loc[3, 'batch-size'] = 16
loggerDf.logsFile.loc[3, 'time-estimated'] = 8100

,date,data-type,net-architecture,loss-func,optim,scheduler,early-stopping-patience,parameters-amount,n-epochs,batch-size,best-train-loss,best-valid-loss,best-kappa,lb-kappa-score,cfg,dataset,trainloop,time_estimated,time-estimated
0,2019.08.11 01:58:43,new_old_mixed_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.CosineAnnealingLR ob...,5.0,28342833.0,11.0,16.0,0.651234,0.631161,0.2751,NaN,from model import MainModel\nfrom torch import...,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,13907.77,NaN
1,2019.08.11 12:36:00,finetuning_new,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.CosineAnnealingLR ob...,10.0,28342833.0,50.0,16.0,0.291845,0.305394,0.8427,NaN,from model import MainModel\nfrom torch import...,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,4296.42,8100.0


In [23]:
loggerDf.save()

In [22]:
datetime.strftime(datetime.now(), "%Y.%m.%d %H:%M:%S")

'2019.08.02 00:51:32'

In [12]:
from libs.lr_finder import LRFinder


In [11]:
import torch

In [12]:
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x16d_wsl')

Downloading: "https://github.com/facebookresearch/WSL-Images/archive/master.zip" to /home/paniquex/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x16-c6f796b0.pth" to /home/paniquex/.cache/torch/checkpoints/ig_resnext101_32x16-c6f796b0.pth
100%|██████████| 777518664/777518664 [01:08<00:00, 11411797.13it/s]


In [13]:
torch.save(model, 'resnext101_32x16d_wsl')

In [37]:
train_new = pd.read_csv('./input/train_new.csv')
train_old = pd.read_csv('./input/train_old.csv')

In [38]:
train_old

,id_code,diagnosis
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
5,15_right,2
6,16_left,4
7,16_right,4
8,17_left,0
9,17_right,1


In [36]:
train_old.to_csv('./input/train_old.csv', index=False)

,id_code,diagnosis
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
5,15_right,2
6,16_left,4
7,16_right,4
8,17_left,0
9,17_right,1
